In [1]:
# Ensure project root (OpenCEP) is on sys.path for imports like `from base.Pattern import Pattern`
import sys, os, pathlib
nb_dir = pathlib.Path(__file__).parent if "__file__" in globals() else pathlib.Path.cwd()
project_root = str((nb_dir / "..").resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from datetime import timedelta
from CEP import CEP
from base.Pattern import Pattern
from base.PatternStructure import SeqOperator, PrimitiveEventStructure
from condition.CompositeCondition import AndCondition
from condition.BaseRelationCondition import SmallerThanCondition
from condition.Condition import Variable, SimpleCondition
from stream.FileStream import FileInputStream, FileOutputStream
from plugin.citibike.CitiBike import CitiBikeTripEventTypeClassifier, CitiBikeDataFormatter
import test

In [3]:
citibikeConsecutiveRidesPattern = Pattern(
    SeqOperator(
        PrimitiveEventStructure("CitiBikeTrip", "a"),
        PrimitiveEventStructure("CitiBikeTrip", "b")),
    AndCondition(
        SimpleCondition(Variable("a", lambda x: (x["start_station_id"], x["end_station_id"])), 
                          Variable("b", lambda x: (x["start_station_id"], x["end_station_id"])),
                          relation_op=lambda a, b: a[1] == b[0])),
    timedelta(minutes=5),
)

In [4]:
cep = CEP([citibikeConsecutiveRidesPattern])
events = FileInputStream("../test/EventFiles/citibike-small.txt")

In [5]:
cep.run(events, FileOutputStream("../test/demo/Matches", "output_citibike.txt"), CitiBikeDataFormatter())

0.807892